<a href="https://colab.research.google.com/github/paulaassisb/projeto-final-ds-awari/blob/main/Projeto_Awari_Paula_Assis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [697]:
## Importação de bibliotecas
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import re
import unicodedata
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
## import matplotlib.pyplot as plt

In [698]:
## Importando arquivo da coleta de dados
url = 'https://raw.githubusercontent.com/paulaassisb/projeto-final-ds-awari/main/coleta_dados_quest_projeto_final.csv'
dados_quest = pd.read_csv(url)
print(f'Número de linhas: {len(dados_quest)}')
print(f'Número de colunas: {len(dados_quest.columns)}')


Número de linhas: 2780
Número de colunas: 21


In [699]:
## Transformando em DataFrame para visualização e manipulação dos dados
df = pd.DataFrame(dados_quest)

In [700]:
df.head()

,Numero do Sujeito,1a. Doença,Idade 1a. Doença,Tempo 1a Doença,Sexo,UF Nascimento,UF,Formação,Atividade,Escolaridade,Estado Civil,Religião,Outro Idioma,Análise do Contexto,Autoconfiança,Autocontrole,Conquistar e Manter Pessoas,Empatia,Leitura Corporal,Otimismo com a Vida,Sentido da Vida
0,13531,depressão,31 - 49 anos,Um ano,Feminino,SP,SP,Administradora,Gerente de Compras,Pós-Graduação,Casado,Católico,Sim,Boa no estresse (PC-P),Fraca no estresse (PC-I),Fraca no estresse (PC-I),Boa diante do impacto do estresse (PC-P),Moderada no estresse (PC-I),Excelente no estresse,Forte no estresse (PC-P),Boa no estresse (PC-P)
1,13532,enfermidade que me levou a 15 dias de hospital...,7 - 12 anos,Mais de um ano,Masculino,SP,SP,Arquiteto,NaN,Pós-Graduação,Casado,Evangélico,Sim,Excelente no estresse,Moderada no estresse (PC-I),Fraca no estresse (PC-I),Boa diante do impacto do estresse (PC-P),Boa no estresse (PC-P),Boa no estresse (PC-P),Excelente no estresse,Moderada no estresse (PC-P)
2,13533,Violência dentro de casa,7 - 12 anos,Mais de um ano,Masculino,SP,SP,NaN,Pesquisa e leitura de livros,Superior Completo,Solteiro,Outro,Não,Boa no estresse (PC-P),Forte no estresse (PC-P),Moderada no estresse (PC-P),Boa diante do impacto do estresse (PC-P),Boa no estresse (PC-P),Moderada no estresse (PC-P),Excelente no estresse,Boa no estresse (PC-P)
3,13534,NaN,NaN,NaN,Masculino,SP,SP,Administração de Empresas,Analista de Mercado,Superior Completo,Solteiro,Evangélico,Sim,Excelente no estresse,Fraca no estresse (PC-I),Fraca no estresse (PC-I),Forte diante do impacto do estresse (PC-P),Excelente no estresse,Boa no estresse (PC-P),Boa no estresse (PC-P),Boa no estresse (PC-P)
4,13535,Conhecer a comunidade quilombola Pedro Cubas,13 - 17 anos,Mais de um ano,Masculino,SP,SP,NaN,NaN,Superior Incompleto,Solteiro,Evangélico,Sim,Excelente no estresse,Excelente face ao estresse,Excelente no estresse,Excelente face ao impacto do estresse (PC-E),Forte no estresse (PC-P),Boa no estresse (PC-P),Excelente no estresse,Excelente no estresse


In [701]:
## Verificação de dados duplicados
duplicados = df.duplicated()
print(duplicados)

0       False
1       False
2       False
3       False
4       False
        ...  
2775    False
2776    False
2777    False
2778    False
2779    False
Length: 2780, dtype: bool



**Transformando as colunas**

In [702]:
df.columns

Index(['Numero do Sujeito', '1a. Doença', 'Idade 1a. Doença',
       'Tempo 1a Doença', 'Sexo', 'UF Nascimento', 'UF', 'Formação',
       'Atividade', 'Escolaridade', 'Estado Civil', 'Religião', 'Outro Idioma',
       'Análise do Contexto', 'Autoconfiança', 'Autocontrole',
       'Conquistar e Manter Pessoas', 'Empatia', 'Leitura Corporal',
       'Otimismo com a Vida', 'Sentido da Vida'],
      dtype='object')

In [703]:
## Nomes das colunas (todos em minúsculo e sem espaço)
df.columns.str.lower()
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [704]:
## Renomear colunas relacionadas a doença/trauma
df = df.rename(columns={'1a._doença': 'trauma_doenca', 'idade_1a._doença': 'idade_doenca_trauma', 'tempo_1a_doença': 'tempo_doenca_trauma', 
                        'formação': 'formacao', 'religião': 'religiao', 'análise_do_contexto': 'analise_do_contexto', 'autoconfiança': 'autoconfianca'})

In [705]:
#Removendo colunas não necessárias
df = df.drop(columns=['numero_do_sujeito', 'atividade'])

**Tratamento de valores nulos (NaN)**

In [706]:
df = df.dropna(subset=['trauma_doenca', 'idade_doenca_trauma', 'tempo_doenca_trauma'])

**Transformando objetos em categorias**

In [707]:
for feature in df.columns:
    print('The feature is {} and number of categories are {}'.format(feature,len(df[feature].unique())))

The feature is trauma_doenca and number of categories are 1722
The feature is idade_doenca_trauma and number of categories are 6
The feature is tempo_doenca_trauma and number of categories are 5
The feature is sexo and number of categories are 2
The feature is uf_nascimento and number of categories are 27
The feature is uf and number of categories are 23
The feature is formacao and number of categories are 721
The feature is escolaridade and number of categories are 7
The feature is estado_civil and number of categories are 5
The feature is religiao and number of categories are 4
The feature is outro_idioma and number of categories are 2
The feature is analise_do_contexto and number of categories are 20
The feature is autoconfianca and number of categories are 22
The feature is autocontrole and number of categories are 24
The feature is conquistar_e_manter_pessoas and number of categories are 22
The feature is empatia and number of categories are 23
The feature is leitura_corporal and 

In [708]:
#Transformando objetos em strings
# label_encoder = LabelEncoder() - não funcionou como esperado, porque classifica por ordem alfabética

escolaridade = {'1ª a 4ª série': 0, '5ª a 9ª série': 1, 'Ensino Médio': 2, 'Técnico': 3, 'Superior Completo': 4, 'Superior Incompleto': 5, 'Pós-Graduação': 6}
df ['escolaridade'] = df.escolaridade.map (escolaridade)

In [709]:
idade_doenca_trauma = {'Até 6 anos': 0, '7 - 12 anos': 1, '13 - 17 anos': 2, '18 - 30 anos':3, '31 - 49 anos': 4, 'Acima': 5}
df ['idade_doenca_trauma'] = df.idade_doenca_trauma.map (idade_doenca_trauma)

In [710]:
tempo_doenca_trauma = {'Alguns dias': 0, 'Semanas': 1, 'Meses': 2, 'Um ano': 3, 'Mais de um ano': 4}
df ['tempo_doenca_trauma'] = df.tempo_doenca_trauma.map (tempo_doenca_trauma)

In [711]:
#Aplicando a moda em valores nulos nas colunas Idade e Tempo de duração da doença ou trauma
df['tempo_doenca_trauma']=df['tempo_doenca_trauma'].fillna(df['tempo_doenca_trauma'].mode()[0])
df['idade_doenca_trauma']=df['idade_doenca_trauma'].fillna(df['idade_doenca_trauma'].mode()[0])

**Padronização do dataframe**


In [712]:
#Padronizando dataframe em minúsculo e sem acentuação

for column in df.columns:
  df[column] = df[column].astype(str).str.lower()
  
cols = df.select_dtypes(include=[np.object]).columns
df[cols] = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))

**Padronizando coluna "Trauma ou Doença"**

In [713]:
stopwords = ['a', 'minha', 'que', 'me', 'de', 'e', 'na', 'quase', 'dos', 'do', 'meus', 'da', 'aos', 'um', 'numa', 'teve', 'enquanto', 'meu', 
             'sem', 'no', 'para', 'primeira', 'com', 'muito', 'fui', 'durante', 'uma', 'em', 'por', 'o', 'quando', 'eu', 'tive']

In [714]:
def removestopword (texto):
  semstop = [p for p in texto.split() if p not in stopwords]
  return ' '.join(semstop)

In [715]:
df['trauma_doenca'] = df['trauma_doenca'].apply(removestopword)

In [716]:
CATEGORIAS = {'acidente': ['acidente', 'acicente', 'queda', 'capotamento', 'atrolepamento', 'batida', 'atropelamento', 'afogamento', 'carro', 'moto',
              'atropelado'],

              'vicios': ['alcool', 'bebida', 'alcolismo', 'alcoolismo', 'drogas', 'vicio'],

              'violencia domestica ou urbana': ['violencia', 'VIOLÊNCIA', 'violência', 'sequestro', 'arma', 'baleado', 'assassinato', 'agressor',
              'assalto', 'assaltada', 'pedofilia', 'estupro', 'abuso', 'abusado'],

              'doencas leves ou virais': ['enfermidade', 'pneumonia', 'tumor', 'labirintite', 'corte', 'cirurgia', 
              'vesicula', 'queimadura', 'fratura', 'chron', 'hernia', 'hepatite', 'disco', 'bronquite', 'apendicite', 'caxumba', 'tireoide',
              'hipertensao', 'torci', 'artrodese', 'gordura', 'lesao', 'sarampo', 'narcolepsia', 'tenorrafia', 'disfuncao', 'condropatia',
              'asma', 'dor', 'gastrite', 'internacao', 'infarto', 'febre', 'meningite', 'obesidade', 'alzhimer', 'anemia', 'neurolofica',
              'neurologico', 'doenca', 'avc', 'visicula', 'ernia', 'cardiaco', 'edema', 'presao', 'tibia', 'renal', 'operacao', 'braco', 'adoecimento',
              'basite', 'lombares', 'dengue', 'coluna', 'menigite', 'audicao', 'infeccao', 'pericardite', 'catarata', ' inflamacao',
              'operei', 'convulsao', 'abcesso', 'artrite', 'trombose', 'catapora', 'infarte', 'epilepsia', 'torcao', 'deslocar', 'inflamacao', 'renais',
              'quebrei', 'tireoide', 'micose', 'clavicula', 'fraturei', 'diabete', 'prielonefrite', 'rcp', 'cisto', 'amputacao', 'toxoplasmose',
              'comvulcao', 'hemorragia', 'derbuculoce', 'deboculose', 'operasao', 'viral', 'virus', 'internado', 'toc', 'traumatismo', 'alergia',
              'enxaqueca', 'apendice', 'verme', 'visual', 'pylori', 'cardiaca', 'garganta', 'cicatriz', 'sifiles', 'bacteria',
              'gagueira', 'salmonela', 'eclampsia', 'retina', 'gripe', 'taquicardia', 'ictiose', 'entalado', 'endometriose', 'denque', 'quebraduras', 
              'quebrado', 'osso', 'quebrar', 'joelho', 'tornozelo'],

              'doencas graves': ['cancer', 'hiv', 'tuberculose',  'cardiopatia', 'esclerose'],

              'divorcio': ['separacao', 'divorcio', 'separaram', 'conjugal', 'relacionamento', 'traicao'],

              'morte': ['falecimento', 'morte', 'falaceu', 'ente', 'perdas', 'perda', 'obito', 'perca', 'perde', 'falecido'],

              'transtornos ou sindromes': ['ansiedade', 'depressao', 'panico', 'estresse', 'traumatico', 'suicidio', 'disturbio', 'perdi', 'mutilacoes',
              'sociais', 'rejeicao', 'burnout', 'bulimia'],

              'gravidez/maternidade': ['gravidez', 'aborto', 'feto'],

              'outros': ['trabalho', 'escola', 'vestibular', 'preso', 'financeiras', 'desemprego', 'desligado', 'interpessoais', 'namoro', 'nenhuma resposta',
                         'nenhuma', 'nenhum', 'nao houve', '2010', 'nao considero', 'nada', 'nao ha', 'nao recordo', 'ca', 'i', 'nao', '']}

In [717]:
CATEGORIAS.keys()

dict_keys(['acidente', 'vicios', 'violencia domestica ou urbana', 'doencas leves ou virais', 'doencas graves', 'divorcio', 'morte', 'transtornos ou sindromes', 'gravidez/maternidade', 'outros'])

In [718]:
def procurar_palavra(lista_palavras, sentenca):
  palavra_presente=False
  for palavra in lista_palavras:
    if palavra in sentenca:
      palavra_presente=True
  return palavra_presente

def mapear_categoria(sentenca):
  for cat in CATEGORIAS.keys():
    if procurar_palavra(CATEGORIAS[cat], sentenca):
      return cat 
  return sentenca

In [719]:
sentenca = 'violencia dentro casa'
mapear_categoria(sentenca)

'violencia domestica ou urbana'

In [720]:
df['trauma_doenca'] = df['trauma_doenca'].apply(mapear_categoria)

**Padronizando coluna "Formação"**

In [721]:
df['formacao'] = df['formacao'].apply(removestopword)

In [722]:
CAT_FORMACAO = {'militar': ['militar', 'molitar', 'bombeito', 'bombeiro', 'bombeira', 'bombeiros', 'oficial', 'defesa civil', 'bm', 'seguranca', 'cbmdf'],
                
                'administracao de empresas': ['administradora', 'administracao', 'administrador', 'adm', 'rh', 'empresarial', 'empresaria', 'aministracao',
                'secretaria', 'empresario', 'escritorio', 'negocios', 'secretariado', 'recursos humanos', 'internacionais', 'gestao', 'exterior', 'gestora'],

                'comunicacao': ['propaganga', 'marketing', 'relacoes publicas', 'comunicacao', 'grafica', 'artes', 'designer', 'publicitario', 'publicitaria',
                'fotografo', 'comunicadora', 'jornalismo', 'telecomunicacoes', 'radiologia', 'jornalista', 'imprensa', 'artes', 'design',
                'publicidade'],
                
                'pedagogia': ['professor', 'professora', 'pedagogia', 'letras', 'pedagogo', 'pedagoga', 'pedagogica', 'pedadogia', 'pedagogica'],

                'ciencias contabeis': ['contador', 'contadora', 'contabeis', 'economicas', 'contabil', 'contabilista'],

                'saude': ['medico', 'medica', 'medicina', 'enfermeiro', 'enfermeira', 'enfermagem', 'nutricionista', 'nutricao', 'neurocirurgiao',
                'farmaceutica', 'clinica', 'biomedicina', 'cirurgia', 'optometria'],
                
                'arquitetura': ['arquiteto', 'arquiteta', 'arquitetura'],

                'ciencias humanas': ['historia', 'geografo', 'geografia', 'filosofia', 'filosofo', 'filosofa', 'sociologo', 'filosifia'],

                'ciencias exatas': ['quimica', 'quimico', 'fisico'],

                'engenharia': ['engenharia', 'engenheiro', 'engenheira', 'eng'],

                'estudante': ['estudante', 'estudanta', 'estudanti', 'estudar', 'escola'],

                'psicologia': ['psicologa', 'psicologo', 'psicologia', 'psicanalista', 'terapia', 'psicolgia'],

                'economia': ['economia', 'financas', 'economista', 'estatistico'],

                'direito': ['advoga', 'advocacia', 'direito', 'advogada', 'advogado'],

                'tecnologia': ['ti', 'informatica', 'computacao', 'redes', 'sistemas', 'informacao', 'dados', 'sistema', 'desenvolvedor', 'programador'],

                'outras areas': ['comercial', 'compradora', 'contas', 'odontologia', 'odontologia', 'testador quest', 'metalurgico', 'ambiental', 'pos',
                'grau', 'escultor', 'turimologa', 'turismologo', 'oceanografa', 'assistente', 'superior', 'seminarista', 'agro',
                'elitricista', 'logistica', 'gastronomia', 'aprendiz', 'mallet', 'operacoes', 'vendas', 'musico', 'coml', 'podologa',
                'manicure', 'corretora', 'bancario', 'gerente', 'massoterapeuta', 'biologo', 'biologia', 'turismo', 'arte', 'mecanico',
                'mecanica', 'fisioterapeuta', 'juridica', 'educador fisico', 'educacao fisica', 'operadora', 'vendedora', 'eletronica',
                'cfo', 'teologia', 'funcionario', 'qualidade', 'teologo', 'condutor.', 'pianista', 'funcionaria', 'nao', 'n', 'nenhuma', 'nenhum',
                'barista', 'biologa', 'cabeleireiro', 'cabelereira', 'coach',  'comprador', 'corretor', 'costureira', 'moda', 'decoradora',
                'doutorado', 'fisiva', 'estagiario', 'farmacia', 'filosifia', 'graduacao', 'graducao', 'hotelaria', 'lavrador', 'maestro', 'magisterio',
                'mba', 'mercadologa', 'mestrado', 'moda', 'motorista', 'musica', 'musicista', 'radialista', 'servidor', 'social', 'taxista', 'tradutora',
                'soldador', 'sim', 'sueprior', 'solteira', '****']}

In [723]:
CAT_FORMACAO.keys()

dict_keys(['militar', 'administracao de empresas', 'comunicacao', 'pedagogia', 'ciencias contabeis', 'saude', 'arquitetura', 'ciencias humanas', 'ciencias exatas', 'engenharia', 'estudante', 'psicologia', 'economia', 'direito', 'tecnologia', 'outras areas'])

In [724]:
def procurar_palavra_form(lista_palavras_form, sentenca_form):
  palavra_presente_form=False
  for palavra_form in lista_palavras_form:
    if palavra_form in sentenca_form:
      palavra_presente_form=True
  return palavra_presente_form

def mapear_categoria2(sentenca_form):
  for categ in CAT_FORMACAO.keys():
    if procurar_palavra_form(CAT_FORMACAO[categ], sentenca_form):
      return categ
  return sentenca_form

In [725]:
sentenca_form = 'administradora'
mapear_categoria2(sentenca_form)

'administracao de empresas'

In [726]:
df['formacao'] = df['formacao'].apply(mapear_categoria2)

**Manipulação de dados - Texto > Números**


In [727]:
#Transformando strings em categorias nominais
label_encoder = LabelEncoder()
df['trauma_doenca'] = label_encoder.fit_transform(df['trauma_doenca'].astype(str))
df['estado_civil'] = label_encoder.fit_transform(df['estado_civil'].astype(str))
df['religiao'] = label_encoder.fit_transform(df['religiao'].astype(str))
df['formacao'] = label_encoder.fit_transform(df['formacao'].astype(str))
df['idade_doenca_trauma'] = label_encoder.fit_transform(df['idade_doenca_trauma'].astype(str))
df['tempo_doenca_trauma'] = label_encoder.fit_transform(df['tempo_doenca_trauma'].astype(str))
df['escolaridade'] = label_encoder.fit_transform(df['escolaridade'].astype(str))

In [728]:
#Transformando categorias nominais
df = pd.get_dummies(df, columns=['sexo'])
df = pd.get_dummies(df, columns=['outro_idioma'])

In [729]:
df.head()

,trauma_doenca,idade_doenca_trauma,tempo_doenca_trauma,uf_nascimento,uf,formacao,escolaridade,estado_civil,religiao,analise_do_contexto,autoconfianca,autocontrole,conquistar_e_manter_pessoas,empatia,leitura_corporal,otimismo_com_a_vida,sentido_da_vida,sexo_feminino,sexo_masculino,outro_idioma_nao,outro_idioma_sim
0,7,4,3,sp,sp,0,6,0,0,boa no estresse (pc-p),fraca no estresse (pc-i),fraca no estresse (pc-i),boa diante do impacto do estresse (pc-p),moderada no estresse (pc-i),excelente no estresse,forte no estresse (pc-p),boa no estresse (pc-p),1,0,0,1
1,3,1,4,sp,sp,1,6,0,2,excelente no estresse,moderada no estresse (pc-i),fraca no estresse (pc-i),boa diante do impacto do estresse (pc-p),boa no estresse (pc-p),boa no estresse (pc-p),excelente no estresse,moderada no estresse (pc-p),0,1,0,1
2,9,1,4,sp,sp,12,4,2,3,boa no estresse (pc-p),forte no estresse (pc-p),moderada no estresse (pc-p),boa diante do impacto do estresse (pc-p),boa no estresse (pc-p),moderada no estresse (pc-p),excelente no estresse,boa no estresse (pc-p),0,1,1,0
4,6,2,4,sp,sp,12,5,2,2,excelente no estresse,excelente face ao estresse,excelente no estresse,excelente face ao impacto do estresse (pc-e),forte no estresse (pc-p),boa no estresse (pc-p),excelente no estresse,excelente no estresse,0,1,0,1
5,2,2,4,sp,sp,5,4,2,0,boa no estresse (pc-p),boa no estresse (pc-p),boa no estresse (pc-p),excelente face ao impacto do estresse (pc-e),boa no estresse (pc-p),boa no estresse (pc-p),moderada no estresse (pc-p),boa no estresse (pc-p),1,0,0,1


In [730]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2103 entries, 0 to 2740
Data columns (total 21 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   trauma_doenca                2103 non-null   int64 
 1   idade_doenca_trauma          2103 non-null   int64 
 2   tempo_doenca_trauma          2103 non-null   int64 
 3   uf_nascimento                2103 non-null   object
 4   uf                           2103 non-null   object
 5   formacao                     2103 non-null   int64 
 6   escolaridade                 2103 non-null   int64 
 7   estado_civil                 2103 non-null   int64 
 8   religiao                     2103 non-null   int64 
 9   analise_do_contexto          2103 non-null   object
 10  autoconfianca                2103 non-null   object
 11  autocontrole                 2103 non-null   object
 12  conquistar_e_manter_pessoas  2103 non-null   object
 13  empatia                      2103

**Cópia dataframe - Teste para apenas uma área da vida (SV)**

Dataframe duplicado com a finalidade de focar apenas na área da vida "Sentido de Vida". Focando nas categorias da escala, será usada a técnica para converter categorias em indicadores.

In [731]:
df_bin = df.copy()
df_bin

,trauma_doenca,idade_doenca_trauma,tempo_doenca_trauma,uf_nascimento,uf,formacao,escolaridade,estado_civil,religiao,analise_do_contexto,autoconfianca,autocontrole,conquistar_e_manter_pessoas,empatia,leitura_corporal,otimismo_com_a_vida,sentido_da_vida,sexo_feminino,sexo_masculino,outro_idioma_nao,outro_idioma_sim
0,7,4,3,sp,sp,0,6,0,0,boa no estresse (pc-p),fraca no estresse (pc-i),fraca no estresse (pc-i),boa diante do impacto do estresse (pc-p),moderada no estresse (pc-i),excelente no estresse,forte no estresse (pc-p),boa no estresse (pc-p),1,0,0,1
1,3,1,4,sp,sp,1,6,0,2,excelente no estresse,moderada no estresse (pc-i),fraca no estresse (pc-i),boa diante do impacto do estresse (pc-p),boa no estresse (pc-p),boa no estresse (pc-p),excelente no estresse,moderada no estresse (pc-p),0,1,0,1
2,9,1,4,sp,sp,12,4,2,3,boa no estresse (pc-p),forte no estresse (pc-p),moderada no estresse (pc-p),boa diante do impacto do estresse (pc-p),boa no estresse (pc-p),moderada no estresse (pc-p),excelente no estresse,boa no estresse (pc-p),0,1,1,0
4,6,2,4,sp,sp,12,5,2,2,excelente no estresse,excelente face ao estresse,excelente no estresse,excelente face ao impacto do estresse (pc-e),forte no estresse (pc-p),boa no estresse (pc-p),excelente no estresse,excelente no estresse,0,1,0,1
5,2,2,4,sp,sp,5,4,2,0,boa no estresse (pc-p),boa no estresse (pc-p),boa no estresse (pc-p),excelente face ao impacto do estresse (pc-e),boa no estresse (pc-p),boa no estresse (pc-p),moderada no estresse (pc-p),boa no estresse (pc-p),1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2735,6,2,2,pr,pr,15,6,2,3,boa face ao estresse (pc-p),excelaancia face ao estresse,forte face ao estresse (pc-p),forte face ao estresse (pc-i),excelaancia face ao estresse,excelaancia face ao estresse,moderada face ao estresse (pc-i),forte face ao estresse (pc- i),1,0,0,1
2736,2,3,2,pr,pr,15,4,2,2,boa face ao estresse (pc-p),boa face ao estresse (pc-i),excelaancia face ao estresse,excelaancia face ao estresse,excelaancia face ao estresse,excelaancia face ao estresse,fraca face ao estresse (pc-i),boa face ao estresse (pc-i),1,0,1,0
2737,5,4,4,pr,pr,15,3,0,0,excelente face ao estresse,boa face ao estresse (pc-p),boa face ao estresse (pc-i),forte face ao estresse (pc-p),boa face ao estresse (pc-p),forte face ao estresse (pc-i),boa face ao estresse (pc-p),excelaancia face ao estresse,1,0,1,0
2738,0,3,2,pr,pr,15,5,0,2,boa face ao estresse (pc-p),boa face ao estresse (pc-p),boa face ao estresse (pc-p),boa face ao estresse (pc-p),boa face ao estresse (pc-p),boa face ao estresse(pc-p),excelente face ao estresse,boa face ao estresse (pc-p),1,0,1,0


In [732]:
SENTIDO_DE_VIDA = {'sv_boa_pcp': ['boa no estresse (pc-p)', 'boa face ao estresse (pc-p)'],
      'sv_boa_pci': ['boa no estresse (pc-i)', 'boa face ao estresse (pc-i)'],
      'equilibrio': ['excelaancia face ao estresse', 'excelente face ao estresse', 'excelente no estresse'],
      'sv_forte_pci': ['forte face ao estresse (pc-i)', 'forte diante do estresse (pc-i)', 'forte no estresse (pc-i)', 'forte face ao estresse (pc- i)'],
      'sv_forte_pcp': ['forte no estresse (pc-p)', 'forte face ao estresse (pc-p)', 'forte diante do estresse (pc-p)'],
      'sv_fraca_pci': ['fraca no estresse (pc-i)', 'fraca face ao estresse (pc-i)', 'fraca diante do estresse (pc-i)'],
      'sv_fraca_pcp': ['fraca no estresse (pc-p)', 'fraca face ao estresse (pc-p)'],
      'sv_moderada_pcp': ['moderada no estresse (pc-p)', 'moderada perante o estresse (pc-p)', 'moderada face ao estresse (pc-p)'],
      'sv_moderada_pci': ['moderada no estresse (pc-i)', 'moderada perante o estresse (pc-i)', 'moderada face ao estresse (pc-i)']}

In [733]:
SENTIDO_DE_VIDA.keys()

dict_keys(['sv_boa_pcp', 'sv_boa_pci', 'equilibrio', 'sv_forte_pci', 'sv_forte_pcp', 'sv_fraca_pci', 'sv_fraca_pcp', 'sv_moderada_pcp', 'sv_moderada_pci'])

In [734]:
def procurar_palavra_sv(lista_palavras_sv, sentenca_sv):
  palavra_presente_sv=False
  for palavra_sv in lista_palavras_sv:
    if palavra_sv in sentenca_sv:
      palavra_presente_sv=True
  return palavra_presente_sv

def mapear_categoria_sv(sentenca_sv):
  for categ_sv in SENTIDO_DE_VIDA.keys():
    if procurar_palavra_sv(SENTIDO_DE_VIDA[categ_sv], sentenca_sv):
      return categ_sv
  return sentenca_sv

In [735]:
sentenca_sv = 'excelente no estresse'
mapear_categoria_sv(sentenca_sv)

'equilibrio'

In [736]:
df_bin['sentido_da_vida'] = df_bin['sentido_da_vida'].apply(mapear_categoria_sv)

In [737]:
df_bin = pd.get_dummies(df_bin, columns=['sentido_da_vida'])

In [738]:
df_bin = df_bin.drop(columns=['uf_nascimento', 'uf', 'analise_do_contexto', 'autoconfianca', 'autocontrole', 'conquistar_e_manter_pessoas',
                              'empatia', 'leitura_corporal', 'otimismo_com_a_vida'])

In [739]:
df_bin.head()

,trauma_doenca,idade_doenca_trauma,tempo_doenca_trauma,formacao,escolaridade,estado_civil,religiao,sexo_feminino,sexo_masculino,outro_idioma_nao,outro_idioma_sim,sentido_da_vida_equilibrio,sentido_da_vida_sv_boa_pci,sentido_da_vida_sv_boa_pcp,sentido_da_vida_sv_forte_pci,sentido_da_vida_sv_forte_pcp,sentido_da_vida_sv_fraca_pci,sentido_da_vida_sv_fraca_pcp,sentido_da_vida_sv_moderada_pci,sentido_da_vida_sv_moderada_pcp
0,7,4,3,0,6,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0
1,3,1,4,1,6,0,2,0,1,0,1,0,0,0,0,0,0,0,0,1
2,9,1,4,12,4,2,3,0,1,1,0,0,0,1,0,0,0,0,0,0
4,6,2,4,12,5,2,2,0,1,0,1,1,0,0,0,0,0,0,0,0
5,2,2,4,5,4,2,0,1,0,0,1,0,0,1,0,0,0,0,0,0
